# Creating Commitment Transactions with HTLC

In this section, we'll create the Commitment Transactions with HTLC in transit from scratch using Python. We'll break down each part of the transaction, explain how it's constructed and signed, and detail the messages exchanged between peers to share the necessary information. Finally, we'll test everything using Bitcoin Core in regtest mode.

## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'
- Lightning Network BOLT #3: '[Commitment Transactions](https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction)'
- Finite Fields, Elliptic Curves and Serialization e.g. [Programming Bitcoin](https://github.com/jimmysong/programmingbitcoin), in particular [Chapter 1](https://github.com/jimmysong/programmingbitcoin/blob/master/ch01.asciidoc), [Chapter2](https://github.com/jimmysong/programmingbitcoin/blob/master/ch02.asciidoc), [Chapter 3](https://github.com/jimmysong/programmingbitcoin/blob/master/ch03.asciidoc) and [Chapter 4](https://github.com/jimmysong/programmingbitcoin/blob/master/ch04.asciidoc).

## HTLC (Hashed Time Locked Contracts)

Once the channel was stablished, the channel can be used to make payments via HTLCs (Hashed Time Locked Contracts).

Changes are sent in batches: one or more `update_ messages` are sent before a `commitment_signed` message. In the following diagram we are presenting just one `update_message` per `commitment_signed` just to keep it simple:

    +-----------+                            +-----------+
    |           |------ update_add_htlc ---->|           |
    |           |----- commitment_signed --->|           |
    |   Alice   |                            |    Bob    |
    |           |<----- revoke_and_ack ------|           |
    |           |                            |           |
    +-----------+                            +-----------+

    The messages are defined in the [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#adding-an-htlc-update_add_htlc).

### The `update_add_htlc` Message

Alice sends the `update_add_htlc`  message to Bob:
- channel_id
- id (htlc counter starting in zero)
- amount_msat
- payment_hash
- cltv_expiry
- onion_routing_packet

### The `commitment_signed` Message

Shortly after sending the `update_add_htlc` message, she will commit to the new state of the channel, so that the HTLC can be safely added by Bob. Bob has the HTLC information and has constructed a new commitment but does not yet have this new commitment signed by Alice.

Alice sends `commitment_signed` to Bob, with the signature for the new commitment and for the HTLC within: 

- channel_id
- signature
- num_htlcs
- htlc_signature

### The `revoke_and_ack` Message

Now that Bob has a new signed commitment, he needs to acknowledge it and revoke the old commitment. He does so by sending the `revoke_and_ack message`:

- channel_id
- per_commitment_secret
- next_per_commitment_point

## Setup
### Requirements

For this exercise we'll run the previous notebook to create the channel funding transaction beetween Alice and Bob.

**You'll need to edit these next line for your local setup.**

In [2]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 2 - Commitment Transactions/First Commitment Transactions.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd9

## Transaction Inputs

Now we can start creating Alice and Bob second commitment transactions. As they are spending the same channel funding transaction, the inputs for all the commitment transactions are almost the same. There only difference comes from the obscured commitment number, remember that to more easily allow both sides to keep track of the commitment numbers, each commitment actually encodes the number of the commitment within the lock time and sequence fields of the commitment transaction. 

In [4]:
# Second Commitment number
commitment_number = 1 

# Obscured commitment number is result of xor operation 
commitment_number_obscured = to_obscure_int_lower48 ^ commitment_number
print("Commitment Number Obscured: " + hex(commitment_number_obscured))

# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (1 byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')
sequence = sequence[::-1]
inputs = (
    channel_funding_txid
    + channel_funding_txindex
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)
print("Inputs: " + inputs.hex())

Commitment Number Obscured: 0xb433fd43a66e
Inputs: bf78b4ba1b702c271a068ed338bebb74ba4a498f60675e073ca2e5292dee2d410100000000fd33b480
